In [685]:
# module Elbmf

using Random, LinearAlgebra, CUDA
using CUDA.CUSPARSE
using CUDA: norm
using Random
using Distributions

In [686]:
A = [1 0 0 1 0;1 0 1 0 1; 0 1 1 0 1; 0 1 0 0 1; 1 1 1 0 1]
U = [1.9 2.5 1.05; 2.2 4.2 0.9; 4.1 0.36 3.1; 5.0 3.1 2.36;1.2 7.002 6.3;]
V = [1.2 6.21 1.09 8.01 2.36; 2.1 7.0 6.36 0.63 1.01; 1.002 9 6.2 5.8 0.05]
X=[2 0.63 1.2 1.2;2.1 0.1 0.6 0.3;6.3 1.2 5.6 0.19]
println("A=",A)
println("U=",U)
println("V=",V)
println("X=",X)

A=[1 0 0 1 0; 1 0 1 0 1; 0 1 1 0 1; 0 1 0 0 1; 1 1 1 0 1]
U=[1.9 2.5 1.05; 2.2 4.2 0.9; 4.1 0.36 3.1; 5.0 3.1 2.36; 1.2 7.002 6.3]
V=[1.2 6.21 1.09 8.01 2.36; 2.1 7.0 6.36 0.63 1.01; 1.002 9.0 6.2 5.8 0.05]
X=[2.0 0.63 1.2 1.2; 2.1 0.1 0.6 0.3; 6.3 1.2 5.6 0.19]


# elastic Net

In [545]:
integrality_gap_elastic(e, kappa, lambda) = 
min(kappa * abs(e) + lambda * e^2, kappa * abs(e - 1) + lambda * (e - 1)^2)

integrality_gap_elastic (generic function with 1 method)

In [546]:
integrality_gap_elastic(2,0.01,0.02)

0.03

# regularizer_elbmf

In [547]:
regularizer_elbmf(x, l1reg, l2reg)= sum(e -> integrality_gap_elastic(e, l1reg, l2reg), x)

regularizer_elbmf (generic function with 1 method)

In [548]:
regularizer_elbmf(X,0.001,0.002)

0.11798600000000001

# proxel_1

In [549]:
proxel_1(x, k, l) = (x <= 0.5 ? (x - k * sign(x)) : (x - k * sign(x - 1) + l)) / (1 + l)

proxel_1 (generic function with 1 method)

In [550]:
proxel_1(4,0.1,0.3)

3.230769230769231

# proxelp

In [551]:
proxelp(x, k, l)= max(proxel_1(x, k, l), zero(x))

proxelp (generic function with 1 method)

In [552]:
proxelp(3, 0.1, 0.2)

2.5833333333333335

# prox_elbmf!

In [553]:
prox_elbmf!(X, k, l)= X .= proxelp.(X, k, l)

prox_elbmf! (generic function with 1 method)

In [554]:
prox_elbmf!(X, 0.01, 0.02)

3×4 Matrix{Float64}:
 1.97059  0.647059   1.18627   1.18627
 2.06863  0.0882353  0.617647  0.284314
 6.18627  1.18627    5.5       0.176471

# proxelb

In [555]:
proxelb(x, k, l)= clamp(proxel_1(x, k, l), zero(x), one(x))

proxelb (generic function with 1 method)

In [556]:
proxelb(2.3,0.01,0.02)

1.0

# prox_elbmf_box

In [557]:
prox_elbmf_box!(X, k, l)= X .= proxelb.(X, k, l)

prox_elbmf_box! (generic function with 1 method)

In [558]:
prox_elbmf_box!(X,0.01,0.02)

3×4 Matrix{Float64}:
 1.0  0.663783   1.0       1.0
 1.0  0.0767013  0.634948  0.268935
 1.0  1.0        1.0       0.163206

# struct Elastic BMF and PALM

In [559]:
mutable struct ElasticBMF{T}
    l1reg::T
    l2reg::T
end

struct PALM end
struct iPALM{T} beta::T end

# rounding

In [560]:
rounding!(fn::ElasticBMF, _, args...) = 
foreach(X -> X .= round.(clamp.(prox_elbmf!(X, 0.5, 1e20), 0, 1)), args)
rounding!(fn, _...)                   = nothing

rounding! (generic function with 2 methods)

In [561]:
fn = ElasticBMF(0.001,0.002)

ElasticBMF{Float64}(0.001, 0.002)

In [562]:
p = PALM()

PALM()

In [563]:
U

5×3 Matrix{Float64}:
 1.9  2.5    1.05
 2.2  4.2    0.9
 4.1  0.36   3.1
 5.0  3.1    2.36
 1.2  7.002  6.3

In [564]:
V

3×5 Matrix{Float64}:
 1.2    6.21  1.09  8.01  2.36
 2.1    7.0   6.36  0.63  1.01
 1.002  9.0   6.2   5.8   0.05

In [565]:
# rounding!(fn,A,U,V)

In [566]:
U

5×3 Matrix{Float64}:
 1.9  2.5    1.05
 2.2  4.2    0.9
 4.1  0.36   3.1
 5.0  3.1    2.36
 1.2  7.002  6.3

In [567]:
V

3×5 Matrix{Float64}:
 1.2    6.21  1.09  8.01  2.36
 2.1    7.0   6.36  0.63  1.01
 1.002  9.0   6.2   5.8   0.05

# apply_rate

In [568]:
apply_rate!(fn::ElasticBMF, fn0, nu) = fn.l2reg = fn0.l2reg * nu
apply_rate!(fn, _...)                = nothing

apply_rate! (generic function with 2 methods)

In [569]:
apply_rate!(fn,fn,2)

0.004

In [570]:
fn.l1reg

0.001

In [571]:
fn.l2reg

0.004

# reducemf_impl PALM

In [572]:
function reducemf_impl!(prox!, ::Type{PALM},  A, U, V)
    VVt, AVt = V * V', A * V'
    grad(x) = x * VVt .- AVt
    L = max(norm(VVt), 1e-4)

    step_size = 1 / (1.1 * L)
    
    U .= U .- grad(U) * step_size
    
    prox!(U, step_size)
end

reducemf_impl! (generic function with 5 methods)

In [573]:
# reducemf_impl!(A,U,V)

In [617]:
U

5×3 Matrix{Float64}:
 1.9  2.5    1.05
 2.2  4.2    0.9
 4.1  0.36   3.1
 5.0  3.1    2.36
 1.2  7.002  6.3

In [618]:
V

3×5 Matrix{Float64}:
 1.2    6.21  1.09  8.01  2.36
 2.1    7.0   6.36  0.63  1.01
 1.002  9.0   6.2   5.8   0.05

# reducemf_impl iPALM

In [687]:
function reducemf_impl!(prox!,opt::iPALM,  A, U, V, U_)
    VVt, AVt = V * V', A * V'
    grad(x) = x * VVt .- AVt
    L = max(norm(VVt), 1e-4)

    @. U = U + opt.beta * (U - U_)
    @. U_ = U

    step_size = 2 * (1 - opt.beta) / (1 + 2 * opt.beta) / L
#     println("grad",grad(U))
#     println("step size",step_size)
    U .= U .- grad(U) * step_size
    prox!(U, step_size)
end

reducemf_impl! (generic function with 5 methods)

In [646]:
# ip=iPALM(0.4)
# U_=deepcopy(U)
# U_

In [647]:
# U

In [648]:
# reducemf_impl!(ip, A, U, V, U_)

In [649]:
# U

In [650]:
# U_

In [651]:
# V

In [581]:
# reducemf_impl!(ip, A, U, V, U_)

# reducemf

In [688]:
reducemf!(fn::ElasticBMF, opt::Type{PALM}, A, U, V)= 
reducemf_impl!((x, alpha) -> prox_elbmf!(x, fn.l1reg * alpha, fn.l2reg * alpha), opt, A, U, V) 

reducemf! (generic function with 2 methods)

# reduceemf _iPALM

In [689]:
reducemf!(fn::ElasticBMF, opt::iPALM, A, U, V, U_)=
reducemf_impl!((x, alpha) -> prox_elbmf!(x, fn.l1reg * alpha, fn.l2reg * alpha), opt, A, U, V, U_) 

reducemf! (generic function with 2 methods)

In [690]:
println("A=",A)
println("U=",U)
println("V=",V)
println("UV=",(U*V))

U_=copy(U)
ip = iPALM(0.2)

reducemf!(fn, ip, A, U, V, U_)

println("A=",A)
println("U=",U)
println("V=",V)
println("UV=",(U*V))

A=[1 0 0 1 0; 1 0 1 0 1; 0 1 1 0 1; 0 1 0 0 1; 1 1 1 0 1]
U=[1.9 2.5 1.05; 2.2 4.2 0.9; 4.1 0.36 3.1; 5.0 3.1 2.36; 1.2 7.002 6.3]
V=[1.2 6.21 1.09 8.01 2.36; 2.1 7.0 6.36 0.63 1.01; 1.002 9.0 6.2 5.8 0.05]
UV=[8.5821 38.749 24.481 22.884 7.0615000000000006; 12.3618 51.162000000000006 34.690000000000005 25.488 9.479000000000001; 8.7822 55.881 25.9786 51.04779999999999 10.194599999999998; 14.87472 73.99 39.798 55.691 15.049; 22.4568 113.166 84.90071999999999 50.56326 10.219019999999999]
A=[1 0 0 1 0; 1 0 1 0 1; 0 1 1 0 1; 0 1 0 0 1; 1 1 1 0 1]
U=[0.20922548202057892 0.8257894351288405 0.0; 0.06206121748191854 1.9586670401624988 0.0; 1.1789761393602283 0.0 0.0; 1.4126980958533277 0.05122686557641868 0.0; 0.0 1.9300963385142196 0.0]
V=[1.2 6.21 1.09 8.01 2.36; 2.1 7.0 6.36 0.63 1.01; 1.002 9.0 6.2 5.8 0.05]
UV=[1.9852283921952598 7.0798162892496785 5.480076582821857 2.196143455116007 1.3278194670486951; 4.18767424531955 14.096069441700205 12.524769102488785 1.7310705873325418 2.1247181838

# factorize palm

In [584]:
function factorize_palm!(
    fn::ElasticBMF,
    X,
    U,
    V,
    regularization_rate,
    maxiter,
    tol;
    callback = nothing
)
    ell = typemax(tol)
    fn0 = deepcopy(fn)

    for t = 1:maxiter
        fn.l2reg = fn0.l2reg * regularization_rate

        reducemf!(fn, PALM, X, U, V)
        reducemf!(fn, PALM, X', V', U')

#         ell, ell0 = norm(X .- U * V)^2, ell

#         (callback !== nothing) && callback((U, V), ell)
#         (abs(ell - ell0) < tol) && break
    end
    fn = fn0
    U, V
end

factorize_palm! (generic function with 1 method)

# factorize iPALM

In [662]:
function factorize_ipalm!(
    fn::ElasticBMF,
    X,
    U,
    V,
    regularization_rate,
    maxiter,
    tol,
    beta;
    callback = nothing,
)
    if beta == 0
        return factorize_palm!(fn, X, U, V, regularization_rate, maxiter, tol; callback=callback)
    end

    ell = typemax(tol)
    fn0 = deepcopy(fn)

    ipalm = iPALM(beta)
    U_    = copy(U)
    Vt_   = copy(V')

    for t = 1:maxiter
        fn.l2reg = fn0.l2reg * regularization_rate

        reducemf!(fn, ipalm, X, U, V, U_)
        reducemf!(fn, ipalm, X', V', U', Vt_)

#         ell, ell0 = norm(X .- U * V)^2, ell

#         (callback !== nothing) && callback((U, V), ell)
#         (abs(ell - ell0) < tol) && break
    end
    fn = fn0
    U, V
end

factorize_ipalm! (generic function with 1 method)

In [540]:
println("A=",A)
println("U=",U)
println("V=",V)
println("UV=",(U*V))

A=[1 0 0 1 0; 1 0 1 0 1; 0 1 1 0 1; 0 1 0 0 1; 1 1 1 0 1]
U=[1.9 2.5 1.05; 2.2 4.2 0.9; 4.1 0.36 3.1; 5.0 3.1 2.36; 1.2 7.002 6.3]
V=[1.2 6.21 1.09 8.01 2.36; 2.1 7.0 6.36 0.63 1.01; 1.002 9.0 6.2 5.8 0.05]
UV=[8.5821 38.749 24.481 22.884 7.0615000000000006; 12.3618 51.162000000000006 34.690000000000005 25.488 9.479000000000001; 8.7822 55.881 25.9786 51.04779999999999 10.194599999999998; 14.87472 73.99 39.798 55.691 15.049; 22.4568 113.166 84.90071999999999 50.56326 10.219019999999999]


# PALM

In [671]:
U,V = factorize_palm!(fn, A, U, V, 0.01, 1000,0.01)

([0.0 1.5719243403963366 0.08600261034479993; 0.3411537896918334 2.824373525748725 0.0; … ; 0.7749345648247725 0.0 1.8657644742611026e-5; 0.7717779148672473 2.3240699665585813 0.004529332896911687], [0.0 1.1005562422902533 … 0.0 1.0814167798121372; 0.4023145302695297 0.0 … 0.020136055258626436 0.13526647328478447; 2.220027864760263 4.56291352497883 … 8.585739135866586 0.0])

In [672]:
println("A=",A)
println("U=",U)
println("V=",V)
println("UV=",(U*V))

A=[1 0 0 1 0; 1 0 1 0 1; 0 1 1 0 1; 0 1 0 0 1; 1 1 1 0 1]
U=[0.0 1.5719243403963366 0.08600261034479993; 0.3411537896918334 2.824373525748725 0.0; 0.987044312834465 0.2208686375108928 0.0; 0.7749345648247725 0.0 1.8657644742611026e-5; 0.7717779148672473 2.3240699665585813 0.004529332896911687]
V=[0.0 1.1005562422902533 0.6532594914044461 0.0 1.0814167798121372; 0.4023145302695297 0.0 0.20326979243077564 0.020136055258626436 0.13526647328478447; 2.220027864760263 4.56291352497883 1.319369623209409 8.585739135866586 0.0]
UV=[0.8233361940333676 0.39242247392577184 0.43299396599489154 0.7700483328046338 0.21262866179592352; 1.1362865083172937 0.37545893282632353 0.7969717714707077 0.0568717413854779 0.750972478736192; 0.08885866215146586 1.086297779907064 0.689691987897201 0.004447423089816869 1.0972824039726448; 4.14204912193943e-5 0.8529442059039246 0.5062579760188707 0.00016018967064973093 0.8380272416579254; 0.9450623621494565 0.8700519562033546 0.9825603320779127 0.08568527198392034 1

In [494]:
rounding!(fn,A,U,V)

In [586]:
println("A=",A)
println("U=",U)
println("V=",V)
println("UV=",(U*V))

A=[1 0 0 1 0; 1 0 1 0 1; 0 1 1 0 1; 0 1 0 0 1; 1 1 1 0 1]
U=[1.9 2.5 1.05; 2.2 4.2 0.9; 4.1 0.36 3.1; 5.0 3.1 2.36; 1.2 7.002 6.3]
V=[1.2 6.21 1.09 8.01 2.36; 2.1 7.0 6.36 0.63 1.01; 1.002 9.0 6.2 5.8 0.05]
UV=[8.5821 38.749 24.481 22.884 7.0615000000000006; 12.3618 51.162000000000006 34.690000000000005 25.488 9.479000000000001; 8.7822 55.881 25.9786 51.04779999999999 10.194599999999998; 14.87472 73.99 39.798 55.691 15.049; 22.4568 113.166 84.90071999999999 50.56326 10.219019999999999]


# iPALM

In [663]:
U,V = factorize_ipalm!(fn, A, U, V, 0.01, 1000,0.01,0.1)


([0.0 0.9895294731510296 0.0; 0.02713317992174029 0.0 0.2809473227871069; … ; 0.3594992435862768 0.0 0.0; 0.2921766577871531 0.1620061955792651 0.19079526849981762], [0.0 2.914127141374017 … 0.0 2.2235042300378627; 1.034316685933416 0.022911900651928135 … 0.9851854130282526 0.0; 3.5769901426165656 0.0 … 0.0 2.823493876372036])

In [664]:
println("A=",A)
println("U=",U)
println("V=",V)
println("UV=",(U*V))

A=[1 0 0 1 0; 1 0 1 0 1; 0 1 1 0 1; 0 1 0 0 1; 1 1 1 0 1]
U=[0.0 0.9895294731510296 0.0; 0.02713317992174029 0.0 0.2809473227871069; 0.36417145146983504 0.0 0.0869870651436886; 0.3594992435862768 0.0 0.0; 0.2921766577871531 0.1620061955792651 0.19079526849981762]
V=[0.0 2.914127141374017 0.9195697369561265 0.0 2.2235042300378627; 1.034316685933416 0.022911900651928135 0.0009129232517713749 0.9851854130282526 0.0; 3.5769901426165656 0.0 3.8338479255128255 0.0 2.823493876372036]
UV=[1.0234868453030121 0.02267200098099118 0.0009033644643526533 0.9748700027099262 0.0; 1.0049458042039958 0.07906953604172791 1.10206016176915 0.0 0.853583785802882; 0.3111518745541192 1.061241910841817 0.6683762250826266 0.0 1.0553442085589866; 0.0 1.0476265030381982 0.330584624860559 0.0 0.7993480888094984; 0.8500385059939549 0.8551517983915999 1.0003047559046163 0.15960614070489415 1.1883653067580535]


In [422]:
rounding!(fn,A,U,V)

In [423]:
println("A=",A)
println("U=",U)
println("V=",V)
println("UV=",(U*V))

A=[1 0 0 1 0; 1 0 1 0 1; 0 1 1 0 1; 0 1 0 0 1; 1 1 1 0 1]
U=[0.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
V=[0.0 1.0 1.0 0.0 1.0; 1.0 0.0 1.0 0.0 1.0; 1.0 1.0 1.0 1.0 1.0]
UV=[0.0 0.0 0.0 0.0 0.0; 1.0 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]


In [38]:
U,V = factorize_palm!(fn, A, U, V, 0.01, 1000,0.01)

([0.0 1.0801994454126125 0.04059031192249591; 0.24951132313880725 1.8329595036359525 0.0; … ; 0.5313443398544986 0.0 0.0; 0.5384180097664464 1.4998539623164966 0.002160509628788274], [0.0 1.5796035475235377 … 0.0 1.6071196341726754; 0.6350448907906517 0.0 … 0.10478279785922728 0.1896978472852204; 1.9991525624790127 7.931733369293641 … 7.6482454184970265 0.0])

In [39]:
U

5×3 Matrix{Float64}:
 0.0       1.0802    0.0405903
 0.249511  1.83296   0.0
 0.659201  0.153895  0.00569898
 0.531344  0.0       0.0
 0.538418  1.49985   0.00216051

In [40]:
V

3×5 Matrix{Float64}:
 0.0       1.5796   0.966943  0.0       1.60712
 0.635045  0.0      0.285521  0.104783  0.189698
 1.99915   7.93173  4.65472   7.64825   0.0

In [41]:
U*V

5×5 Matrix{Float64}:
 0.767121  0.321952  0.497356  0.423631   0.204912
 1.16401   0.394129  0.764611  0.192063   0.748703
 0.109123  1.08648   0.707877  0.0597128  1.08861
 0.0       0.839313  0.51378   0.0        0.853934
 0.956794  0.867624  0.958915  0.173683   1.14982

In [203]:
rounding!(fn,A,U,V)

In [43]:
println("A=",A)
println("U=",U)
println("V=",V)
println("UV=",(U*V))

A=[1 0 0 1 0; 1 0 1 0 1; 0 1 1 0 1; 0 1 0 0 1; 1 1 1 0 1]
U=[0.0 1.0 0.0; 0.0 1.0 0.0; 1.0 0.0 0.0; 1.0 0.0 0.0; 1.0 1.0 0.0]
V=[0.0 1.0 1.0 0.0 1.0; 1.0 0.0 0.0 0.0 0.0; 1.0 1.0 1.0 1.0 0.0]
UV=[1.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0; 0.0 1.0 1.0 0.0 1.0; 0.0 1.0 1.0 0.0 1.0; 1.0 1.0 1.0 0.0 1.0]


In [44]:
A-U*V

5×5 Matrix{Float64}:
 0.0  0.0   0.0  1.0  0.0
 0.0  0.0   1.0  0.0  1.0
 0.0  0.0   0.0  0.0  0.0
 0.0  0.0  -1.0  0.0  0.0
 0.0  0.0   0.0  0.0  0.0

In [ ]:
U = [1 2 4;4,1,0]
U = 